In [1]:
import pandas as pd
from ragas import evaluate as rag_eval
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
from datasets import Dataset
import evaluate as eval
from concurrent.futures import ThreadPoolExecutor
import os

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Finding out the Basic QA Metrics (F1 score, EM score)

In [ ]:
squad_metric = eval.load("squad")

df = pd.read_csv('../results/rag_generated_answers.csv')

predictions = []
references = []

# Loop through each row of the DataFrame
for index, row in df.iterrows():
    # Create a unique ID for each row. Using the index is a simple way.
    unique_id = str(index)

    # Format the prediction dictionary
    predictions.append({
        'id': unique_id,
        'prediction_text': row['generated_answer']
    })

    # Format the reference dictionary
    answer = row['answer']
    context = row['top_1_context']

    # Calculate the start index of the answer in the context
    answer_start = context.find(answer)

    references.append({
        'id': unique_id,
        'answers': {
            'text': [answer],                   # Must be a list of strings
            'answer_start': [answer_start]      # Must be a list of integers
        }
    })

squad_metric = eval.load("squad")
results = squad_metric.compute(predictions=predictions, references=references)

print("Evaluation Results:")
print(f"EM: {results['exact_match']:.2f}\nF1: {results['f1']:.2f}")

Evaluation Results:
EM: 34.86
F1: 43.13


# Advanced Evaluation using RAGAs

In [ ]:
# Limit to 200 queries for RAGAS evaluation
df_subset = df.head(200)

data = {
    "question": df_subset['question'].tolist(),           # List of questions from the dataset
    "answer": df_subset['generated_answer'].tolist(),     # List of generated answers
    "retrieved_contexts": df_subset['top_1_context'].apply(lambda x: [x]).tolist(),      # List of top-1 contexts
    "reference": df_subset['answer'].tolist()         # List of ground truth answers (human-annotated)
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)
print(f"Dataset size: {len(dataset)} samples")

Dataset size: 200 samples


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"

In [5]:
# Function to evaluate a subset of the dataset
def evaluate_subset(subset_dataset, metrics):
    """Evaluate a subset of the dataset using RAGAS metrics"""
    try:
        result = rag_eval(
            dataset=subset_dataset,
            metrics=metrics
        )
        return result.to_pandas()
    except Exception as e:
        print(f"Error in evaluation: {e}")
        return None

# Split dataset into chunks for multiprocessing
def split_dataset_into_chunks(dataset, chunk_size=50):
    """Split dataset into smaller chunks for parallel processing"""
    chunks = []
    for i in range(0, len(dataset), chunk_size):
        chunk_data = {
            "question": dataset["question"][i:i+chunk_size],
            "answer": dataset["answer"][i:i+chunk_size],
            "retrieved_contexts": dataset["retrieved_contexts"][i:i+chunk_size],
            "reference": dataset["reference"][i:i+chunk_size]
        }
        chunk_dataset = Dataset.from_dict(chunk_data)
        chunks.append(chunk_dataset)
    return chunks

# Define metrics
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
]

# Split dataset into chunks (50 samples per chunk for 200 total samples = 4 chunks)
dataset_chunks = split_dataset_into_chunks(dataset, chunk_size=50)
print(f"Split dataset into {len(dataset_chunks)} chunks of ~50 samples each")

# Use ThreadPoolExecutor for parallel evaluation
n_workers = 4  # Adjust based on your system capabilities
print(f"Using {n_workers} workers for parallel evaluation")

with ThreadPoolExecutor(max_workers=n_workers) as executor:
    # Submit all evaluation tasks
    future_results = [
        executor.submit(evaluate_subset, chunk, metrics) 
        for chunk in dataset_chunks
    ]
    
    # Collect results
    results_list = []
    for i, future in enumerate(future_results):
        try:
            result = future.result(timeout=300)  # 5 minute timeout per chunk
            if result is not None:
                results_list.append(result)
                print(f"Completed evaluation for chunk {i+1}")
            else:
                print(f"Failed evaluation for chunk {i+1}")
        except Exception as e:
            print(f"Error in chunk {i+1}: {e}")

# Combine all results
if results_list:
    df_results = pd.concat(results_list, ignore_index=True)
    print(f"Combined results shape: {df_results.shape}")
    print(df_results.head())
else:
    print("No successful evaluations completed")

Split dataset into 4 chunks of ~50 samples each
Using 4 workers for parallel evaluation


Evaluating:   0%|          | 0/200 [00:00<?, ?it/s]


Exception raised in Job[6]: APIConnectionError(Connection error.)


Exception raised in Job[11]: APIConnectionError(Connection error.)
Exception raised in Job[14]: APIConnectionError(Connection error.)
Exception raised in Job[15]: APIConnectionError(Connection error.)
Exception raised in Job[16]: APIConnectionError(Connection error.)


Exception raised in Job[11]: APIConnectionError(Connection error.)

Exception raised in Job[12]: APIConnectionError(Connection error.)

Exception raised in Job[20]: APIConnectionError(Connection error.)


Exception raised in Job[5]: APIConnectionError(Connection error.)


Exception raised in Job[2]: APIConnectionError(Connection error.)

Exception raised in Job[1]: APIConnectionError(Connection error.)
Exception raised in Job[5]: APIConnectionError(Connection error.)
Evaluating:   0%|          | 1/200 [00:09<31:09,  9.40s/it]Exception raised in Job[1]: APIConnectionError(Connection error.)
Exception r

Completed evaluation for chunk 1


Exception raised in Job[19]: TimeoutError()
Exception raised in Job[36]: TimeoutError()

Exception raised in Job[128]: APIConnectionError(Connection error.)
Exception raised in Job[48]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-JMOQL8oB4yQNxEFbGkMUDdzj on tokens per min (TPM): Limit 200000, Used 199910, Requested 463. Please try again in 111ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}})

Exception raised in Job[181]: APIConnectionError(Connection error.)

Exception raised in Job[53]: TimeoutError()
Exception raised in Job[172]: APIConnectionError(Connection error.)

Exception raised in Job[179]: RateLimitError(Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-JMOQL8oB4yQNxEFbGkMUDdzj on tokens per min (TPM): Limit 200000, Used 200000, Requested 1100. Please try again in 330ms. Vis

Completed evaluation for chunk 2
Completed evaluation for chunk 3
Completed evaluation for chunk 4
Combined results shape: (200, 8)
                                          user_input  \
0  Was Abraham Lincoln the sixteenth President of...   
1  Did Lincoln sign the National Banking Act of 1...   
2                   Did his mother die of pneumonia?   
3      How many long was Lincoln's formal education?   
4       When did Lincoln begin his political career?   

                                  retrieved_contexts  \
0                            [Young Abraham Lincoln]   
1  [Lincoln believed in the Whig theory of the pr...   
2  [An autopsy performed after his death revealed...   
3  [Lincoln's formal education consisted of about...   
4  [Lincoln began his political career in 1832, a...   

                    response  reference  faithfulness  answer_relevancy  \
0                       yes.        yes           NaN               NaN   
1  House of Representatives.        yes     

In [6]:
df_results.to_csv("../results/rag_evaluation_results_multiprocessing.csv", index=False)

In [7]:
mean_scores = df_results[["faithfulness", "answer_relevancy", "context_precision", "context_recall"]].mean()
print("Mean Scores:")
print(mean_scores)

Mean Scores:
faithfulness        NaN
answer_relevancy    NaN
context_precision   NaN
context_recall      NaN
dtype: float64
